<a href="https://colab.research.google.com/github/andrewwgordon/lang-chain-testbed/blob/main/self_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install lark
!pip install chromadb